<a href="https://colab.research.google.com/github/karasu1982/POS_Data_Analytics/blob/main/notebook/%E3%83%90%E3%82%B9%E3%82%B1%E3%83%83%E3%83%88%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# バスケット分析

バスケット分析は、１回の来店時に同時に購入される商品を分析する手法で、「ビールとオムツ」の例（実はあれば、嘘だったという説もありますが）で非常に有名になりました。

ビールとオムツの真偽はともかく、「ある商品Aを買った人に別の商品Bを勧めると購入されやすくなる」という発想は、小売業のMDやインストアマーケティングとも合致して、様々な場面で使われています。
最も分かり易いのは、棚割りでしょうか。上記の例でいうと、商品AとBを近くに並べることで、どちらかを購入した人がもう一方も購入することを期待できます。

---

このように、分かり易く・効果につながりやすい分析ですが、２点気を付ける必要があります。

① 当たり前の組み合わせに騙されない。


例えば、「カレーのルーと一緒に、人参やジャガイモが購入されていた」という結果が出たとき、カレーをつくって食べる家庭の数がわかっても、それ以外に何か追加購入を促すことは難しいでしょう。


② 理由が説明できない組み合わせには注意。

例えば、「スイカと芳香剤が同時購入されていた」という結果が出たとします。

まず、特別関係がなさそうに見えて、「なぜだろう？」と思いますよね。
また、あなたがスーパーマーケットの店長さんだとして、分析結果がそうでも、スイカ（食品）の横に、芳香剤って置くでしょうか。逆に、どちらの売上も下がりそうですよね。

このような、理由が説明できなかったり・一般的に見て有り得ない組み合わせの場合、どう活用するかは注意が必要です。

棚割りには使えないかもしれないけど、同時購入での割引クーポンなら使えそうとか。そういうことを考えてみても面白いかもしれませんね。

## 環境設定

In [ ]:
%%bash
pip install duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.1 MB/s eta 0:00:00


In [ ]:
# 標準ライブラリ
import pandas as pd
import numpy as np

# データ見える化
import seaborn as sns
import matplotlib.pyplot as plt

# DuckDB
import duckdb

## データ準備

ID-POSのサンプルデータとして、下記を利用

https://www.kyoritsu-pub.co.jp/book/b10003634.html

In [ ]:
%%bash
wget https://kyoritsu-pub.sakura.ne.jp/app/file/goods_contents/2319.zip
unzip /content/2319.zip
unzip /content/DataProcess.20151001/src/data/Tafeng/Tafeng.zip

In [ ]:
df = pd.read_csv("/content/Tafeng_dataset/Tafeng.csv")

# データ型を
df = df.astype({'CustID': 'object', 'ProductSubClass': 'object', 'ProductID': 'object'})

In [ ]:
df.head(3)

,Time,CustID,Age,Area,ProductSubClass,ProductID,Amount,Asset,SalesPrice
0,2000-11-01 00:00:00,46855,D,E,110411,4710085120468,3,51,57
1,2000-11-01 00:00:00,539166,E,E,130315,4714981010038,2,56,48
2,2000-11-01 00:00:00,663373,F,E,110217,4710265847666,1,180,135


## データ集計

In [ ]:
import duckdb

# SQLに慣れている方にとっては、少々くどいかもしれませんが、可読性を高めるために、処理を１つ１つ分解してWITH句で実行しています。
#
# 私がBigQueryの文法に慣れていることもあり、次の3点で修正が必要でした。他にも微妙な差はありそうなので、都度都度直す必要がありますね。
# ・コメントアウトは#は使わず、-- で行う
# ・文字列はダブルコーテーション（ " )ではなく、シングルクオーテーション（ ' )でくくる
# ・日付項目は、文字列を自動で日付型化しないため、DATETIME '2000-11-01'のように日付型を明示的に指定する
#

df_output = duckdb.query(
    f"""
    WITH
    t_all AS(
      -- 総来店者数
      SELECT
       COUNT(DISTINCT CustID) AS Num_of_All
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
    ),

    t_purchaser AS(
      -- 商品ごとの購入者数
      SELECT
        CAST(ProductID AS STRING) AS ProductID, COUNT(DISTINCT CustID) AS Num_of_Purchaser
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
      GROUP BY ProductID
    ),

    t_simultaneous_purchaser_pre AS(
      -- 商品ごとの同時購入者
      SELECT
        DISTINCT CAST(ProductID AS STRING) AS ProductID, CAST(Time AS DATE) Purchase_date, CustID
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
    ),

    t_simultaneous_purchaser AS(
      -- 商品ごとの同時購入者数
      SELECT
        t1.ProductID as ProductID_A, t2.ProductID as ProductID_B, COUNT(DISTINCT CustID) as Num_of_Simultaneous_Purchaser
      FROM t_simultaneous_purchaser_pre as t1
      INNER JOIN t_simultaneous_purchaser_pre as t2
      USING(Purchase_date, CustID)
      WHERE t1.ProductID != t2.ProductID
      GROUP BY t1.ProductID, t2.ProductID
    )

    SELECT
      ProductID_A,
      t_purchaser.Num_of_Purchaser AS Num_of_Purchaser_A,
      t_purchaser.Num_of_Purchaser / Num_of_All AS PurchaseRate_A,
      ProductID_B,
      t2.Num_of_Purchaser AS Num_of_Purchaser_B,
      t2.Num_of_Purchaser / Num_of_All AS PurchaseRate_B,
      Num_of_Simultaneous_Purchaser,
      Num_of_Simultaneous_Purchaser / t_purchaser.Num_of_Purchaser AS CombinedSalesRate,
      (Num_of_Simultaneous_Purchaser / t_purchaser.Num_of_Purchaser) / (t2.Num_of_Purchaser / Num_of_All) AS Lift

    FROM t_purchaser
    LEFT OUTER JOIN t_simultaneous_purchaser
    ON t_purchaser.ProductID = t_simultaneous_purchaser.ProductID_A

    LEFT OUTER JOIN t_purchaser as t2
    ON t_simultaneous_purchaser.ProductID_B = t2.ProductID

    FULL OUTER JOIN t_all
    ON True
    """
).to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

作成したテーブルのアウトプット
*   ProductID_A ： 基準となる商品の商品コード（商品A）
*   Num_of_Purchaser_A ： 商品Aの購入者数
*   PurchaseRate_A ： 商品Aの購入率（商品Aの購入者数÷総来店者数）
*   ProductID_B ： 同時購入対象の商品の商品コード（商品B）
*   Num_of_Purchaser_B ：商品Bの購入者数
*   PurchaseRate_B ： 商品Bの購入率（商品Bの購入者数÷総来店者数）
*   Num_of_Simultaneous_Purchaser：商品Aと商品Bの同時購入者数
*   CombinedSalesRate：同時購入率（併売率）（商品Aと商品Bの同時購入者数÷商品Aの総購入者数）
*   Lift：リフト値　同時購入率（併売率）（商品Aと商品Bの同時購入者数÷商品Aの総購入者数）÷商品Bの購入率（商品Bの購入者数÷総来店者数）

同時購入者数・同時購入率が多い順にソートします。すると、トップの商品の組み合わせ（商品A:4710421090059と商品B:4711271000014）は、同時購入率が0.331(33.1%)と、商品B全体の購入率18.7%よりも高くなっています。

同時購入率 ÷ B全体の購入率で計算できるリフト値は1.77となっており、商品Bを購入する確率よりも、商品A購入者がBを購入する確率（同時購入率）の方が1.7倍高くなるといえます。

In [ ]:
 # データ確認
df_output.sort_values(["Num_of_Simultaneous_Purchaser","CombinedSalesRate"], ascending = False)

,ProductID_A,Num_of_Purchaser_A,PurchaseRate_A,ProductID_B,Num_of_Purchaser_B,PurchaseRate_B,Num_of_Simultaneous_Purchaser,CombinedSalesRate,Lift
1008466,4710421090059,1235,0.052355,4711271000014,4404.0,0.186697,409.0,0.331174,1.773857
1040291,4711271000014,4404,0.186697,4710421090059,1235.0,0.052355,409.0,0.092870,1.773857
3725578,4710683100015,647,0.027428,4711271000014,4404.0,0.186697,289.0,0.446677,2.392521
984757,4711271000014,4404,0.186697,4710683100015,647.0,0.027428,289.0,0.065622,2.392521
3883048,4710011401135,369,0.015643,4710011401128,636.0,0.026962,273.0,0.739837,27.440290
...,...,...,...,...,...,...,...,...,...
3915003,NaN,1,0.000042,NaN,NaN,NaN,NaN,NaN,NaN
3915004,NaN,1,0.000042,NaN,NaN,NaN,NaN,NaN,NaN
3915005,NaN,1,0.000042,NaN,NaN,NaN,NaN,NaN,NaN
3915006,NaN,1,0.000042,NaN,NaN,NaN,NaN,NaN,NaN
